# Generate Noun Property Ratings

This notebook rates all nouns on selected properties and saves to JSON/JSONL.

**Output:** 
- `noun_property_ratings.jsonl` - JSONL format (one JSON object per line)
- `noun_property_nested.json` - Nested JSON format (nouns with property objects)

## 1. Setup

In [1]:
import torch
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from tqdm import tqdm
import os
import re
import json

/Users/b/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## 2. Configuration

In [2]:
# File paths
NOUNS_FILE = "nouns.txt"
PROPERTIES_FILE = "curated_properties.txt"  # Use curated_properties.txt for 15 properties
OUTPUT_FILE = "noun_curated_property_ratings.jsonl"  # JSONL format (one JSON object per line)

# How often to save progress
SAVE_FREQUENCY = 200

## 3. Load Model

In [3]:
print("Loading model...")

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="mps",  # Change to "cuda" for NVIDIA GPU or "cpu" for CPU, mps for mac
    torch_dtype="auto",
    trust_remote_code=False,
)

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=50,
    do_sample=False,
)

print("Model loaded!")

`torch_dtype` is deprecated! Use `dtype` instead!


Loading model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use mps
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Model loaded!


## 4. Load Nouns and Properties

In [4]:
# Read nouns
with open(NOUNS_FILE, 'r') as f:
    nouns = [line.strip() for line in f if line.strip()]

# Read properties
with open(PROPERTIES_FILE, 'r') as f:
    properties = [line.strip() for line in f if line.strip()]

print(f"Loaded {len(nouns)} nouns")
print(f"Loaded {len(properties)} properties")
print(f"Total ratings needed: {len(nouns) * len(properties):,}")
print(f"Estimated time: {len(nouns) * len(properties) * 2 / 3600:.1f} hours")

Loaded 1852 nouns
Loaded 15 properties
Total ratings needed: 27,780
Estimated time: 15.4 hours


## 5. Rating Function

In [5]:
def rate_noun_on_property(noun, property_name, pipe):
    """
    Rate a noun on a property from 1-10.
    Returns -1 if rating extraction fails.
    """
    prompt = f"""Rate the {property_name} of '{noun}' from 1 to 10.
1 = very low {property_name}
10 = very high {property_name}
Respond with only a number 1-10."""
    
    messages = [{"role": "user", "content": prompt}]
    
    try:
        output = pipe(messages)
        response = output[0]["generated_text"].strip()
        
        # Extract number from response
        numbers = re.findall(r'\b([1-9]|10)\b', response)
        if numbers:
            rating = int(numbers[0])
            if 1 <= rating <= 10:
                return rating
        
        return -1
    except Exception as e:
        print(f"Error: {noun} - {property_name}: {str(e)}")
        return -1

## 6. Test Rating Function

In [6]:
# Quick test
print("Testing...")
test_rating = rate_noun_on_property("elephant", "size", pipe)
print(f"elephant - size: {test_rating}/10")

Testing...
elephant - size: 10/10


## 7. Generate All Ratings

In [7]:
# Check for existing progress
if os.path.exists(OUTPUT_FILE):
    existing_ratings = []
    with open(OUTPUT_FILE, 'r') as f:
        for line in f:
            existing_ratings.append(json.loads(line))
    completed = set((r['noun'], r['property']) for r in existing_ratings)
    print(f"Found {len(existing_ratings)} existing ratings")
else:
    existing_ratings = []
    completed = set()
    print("Starting fresh")

# Create list of remaining work
all_combinations = [(noun, prop) for noun in nouns for prop in properties]
remaining = [combo for combo in all_combinations if combo not in completed]

print(f"Remaining: {len(remaining):,} ratings")
print("\nStarting...\n")

Found 13800 existing ratings
Remaining: 13,980 ratings

Starting...



In [8]:
# Main loop
results = []

for i, (noun, prop) in enumerate(tqdm(remaining, desc="Rating")):
    # Get rating
    rating = rate_noun_on_property(noun, prop, pipe)
    
    # Store
    results.append({
        'noun': noun,
        'property': prop,
        'rating': rating
    })
    
    # Save periodically
    if (i + 1) % SAVE_FREQUENCY == 0:
        # Append new results to file
        with open(OUTPUT_FILE, 'a') as f:
            for result in results:
                f.write(json.dumps(result) + '\n')
        results = []
        print(f"Saved at {i + 1} ratings")

# Final save
if results:
    with open(OUTPUT_FILE, 'a') as f:
        for result in results:
            f.write(json.dumps(result) + '\n')

print(f"\nDone! Saved to {OUTPUT_FILE}")

Rating:   1%|▏         | 200/13980 [02:45<8:32:09,  2.23s/it] 

Saved at 200 ratings


Rating:   3%|▎         | 400/13980 [05:22<2:03:24,  1.83it/s] 

Saved at 400 ratings


Rating:   4%|▍         | 600/13980 [07:53<1:49:16,  2.04it/s] 

...

Saved at 8000 ratings


Rating:  59%|█████▊    | 8200/13980 [1:38:09<1:00:26,  1.59it/s]

Saved at 8200 ratings


Rating:  59%|█████▊    | 8208/13980 [1:38:13<1:09:04,  1.39it/s]


KeyboardInterrupt: 

## 8. View Results

In [ ]:
# Load and display results
ratings = []
with open(OUTPUT_FILE, 'r') as f:
    for line in f:
        ratings.append(json.loads(line))

print(f"Total ratings: {len(ratings):,}")
failed = sum(1 for r in ratings if r['rating'] == -1)
print(f"Failed ratings: {failed}")
print("\nFirst 20 ratings:")
for rating in ratings[:20]:
    print(rating)

In [ ]:
# Create nested format (optional - easier to work with)
# Structure: { noun: { property: rating, ... }, ... }
nested = {}
for rating in ratings:
    if rating['rating'] != -1:  # Skip failed ratings
        noun = rating['noun']
        if noun not in nested:
            nested[noun] = {}
        nested[noun][rating['property']] = rating['rating']

with open('noun_property_nested.json', 'w') as f:
    json.dump(nested, f, indent=2)

print("Also saved nested format to: noun_property_nested.json")
print(f"Structure: {len(nested)} nouns with properties as nested objects")